In [365]:
import warnings
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

warnings.filterwarnings("ignore", category=UserWarning)

In [366]:
import torch
import numpy as np
from transformers import BertTokenizerFast, BertForMultipleChoice, Trainer, TrainingArguments
from datasets import load_dataset


In [367]:
# Загружаем предварительно подготовленный датасет
ds = load_dataset("PARus")

model_name = "DeepPavlov/rubert-base-cased"
# Загружаем предобученную модель
model = BertForMultipleChoice.from_pretrained(model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    model = model.to("cuda")
# Подготавливаем токенизатор
tokenizer = BertTokenizerFast.from_pretrained(model_name)

max_length_choice1 = max([len(dict(i).get('choice1')) for i in ds['train']])
max_length_choice2 = max([len(dict(i).get('choice2')) for i in ds['train']])
max_length = max(max_length_choice1, max_length_choice2)
max_length

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


64

In [368]:
# Функция подготовки выборок для multiple choice task
def prepare_features(batch):
    # Конструируем prompt для каждого варианта выбора
    choices = [batch['choice1'], batch['choice2']]
    first_sentences = [batch['premise']] * len(choices)
    second_sentences = [f"{batch['question']} {choice}" for choice in choices]

    # Токенизируем с сохранением связи индексов
    inputs = tokenizer(first_sentences,
                       second_sentences,
                       truncation="only_second",
                       max_length=max_length,
                       padding="max_length"
                       )

    # Добавляем метку правильного ответа
    label = batch.get('label')  # Если label отсутствует, значит тестовая выборка
    if label is not None:
        inputs['labels'] = label
    return inputs

In [369]:
# Обрабатываем датасеты
for split_name in ['train', 'validation']:
    ds[split_name] = ds[split_name].map(
        prepare_features,
        batched=False,
        remove_columns=['premise', 'choice1', 'choice2', 'question', 'label']
    )

ds["validation"] = ds["validation"].shuffle()
ds["train"] = ds["train"].shuffle()

In [370]:
# Преобразуем объект в PyTorch TensorDataset
ds["train"].set_format(type='torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'labels'])
ds["validation"].set_format(type='torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'labels'])

ds["train"]

Dataset({
    features: ['idx', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 400
})

In [371]:
# Определение метрик точности (accuracy)
from sklearn.metrics import accuracy_score


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=1)
    return {"accuracy": accuracy_score(labels, predictions)}

In [372]:
# Аргументы обучения
training_args = TrainingArguments(
    output_dir='./rubert_finetuned_PARus',
    eval_strategy="epoch",
    save_strategy='epoch',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=4,
    learning_rate=1e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    greater_is_better=True,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

# Создаем тренера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds['train'],
    eval_dataset=ds['validation'],
    compute_metrics=compute_metrics,
)

In [373]:
# Начинаем обучение
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.691794,0.580000
2,No log,0.691098,0.600000
3,No log,0.688724,0.600000
4,No log,0.686179,0.620000
5,No log,0.685087,0.610000


TrainOutput(global_step=125, training_loss=0.6868722534179688, metrics={'train_runtime': 1408.0339, 'train_samples_per_second': 1.42, 'train_steps_per_second': 0.089, 'total_flos': 131554346496000.0, 'train_loss': 0.6868722534179688, 'epoch': 5.0})

In [374]:
# Оцениваем модель на валидирующем наборе
eval_results = trainer.evaluate()
print(f'Точность в наборе для проверки: {eval_results["eval_accuracy"]:.3f}')


Точность в наборе для проверки: 0.620


In [375]:
# Сохранение финальной модели
trainer.save_model("./rubert_finetuned_PARus/final_model")